# Slicing

In [ ]:
# import
import sys
sys.path.append('../')

import logging
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

# Set up the logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [ ]:
mri_1_label = nib.load('..\MRI\Labels\labels_00001.nii')
mri_1_label_data = mri_1_label.get_fdata()
mri_1_label_data.shape

In [ ]:
# Visualize an image slice using simpleITK
import SimpleITK as sitk

# Load the image data
volume = sitk.ReadImage('..\MRI\Labels\labels_00001.nii')
img_array = sitk.GetArrayFromImage(volume)

# Take a 2D image slice
slice_num = 670
slice = img_array[:, slice_num, :]
plt.imshow(slice, aspect=6)
plt.show()

In [ ]:
volume_size = volume.GetSize()
img_size = img_array.shape

print(f" {volume_size=},\n {img_size=}")

In [ ]:
# Euler transformation
# Rotation is defined by three rotations around z1, x2, z2 axis
th_z1 = np.deg2rad(20)
th_x2 = np.deg2rad(0)

# Translation vector
o = np.array(volume.GetOrigin())

# transformation simplified at z2=0 since this rotation is never performed
eul_tr = np.array([ [np.cos(th_z1), -np.sin(th_z1)*np.cos(th_x2),  np.sin(th_z1)*np.sin(th_x2), o[0]],
                    [np.sin(th_z1),  np.cos(th_z1)*np.cos(th_x2), -np.cos(th_z1)*np.sin(th_x2), o[1]+140],
                    [0,              np.sin(th_x2),                np.cos(th_x2),               o[2]],
                    [0,              0,                            0,                           1]])


In [ ]:
# Define plane's coordinate system
e1 = eul_tr[0][:3] # x-coordinate of image plane
e2 = eul_tr[1][:3] # y-coordinate of image plane
e3 = eul_tr[2][:3] # normal vector of image plane
origin = eul_tr[:, -1:].flatten()[:3] # origin of the image plane

print(f" {e1=},\n {e2=},\n {e3=},\n {origin=}")

# Direction for the resampler will be (e1, e2, e3) flattened
direction = np.stack([e1, e2, e3], axis=0).flatten()
print(f" {direction=}")

In [ ]:
# Define the size and resolution of the output image
# height of the image plane: original z size divided by the z component of the normal vector
# (cosine of the angle between the normal vector and the z axis: x-rotation)
h = int(abs(volume_size[2]//e3[2]))
# width of the image plane: original x size divided by the x component of the normal vector
# (cosine of the angle between the normal vector and the x axis: z-rotation) 
w = int(abs(volume_size[0]//e1[0]))


print(f" {h=},\n {w=}") 

In [ ]:
# Use SimpleITK's resampler
resampler = sitk.ResampleImageFilter()
# Extract properties from the SimpleITK Image
spacing = volume.GetSpacing()

# use reference image
# resampler.SetReferenceImage(volume)
resampler.SetOutputDirection(direction.tolist())
resampler.SetOutputOrigin(origin.tolist())
resampler.SetOutputSpacing(spacing)
resampler.SetSize((w, 3, h))
resampler.SetInterpolator(sitk.sitkNearestNeighbor)

In [ ]:
# Resample the volume on the arbitrary plane
sliced_volume = resampler.Execute(volume)

# Convert the image to a numpy array
sliced_img = sitk.GetArrayFromImage(sliced_volume)
# print value range of slice
print(f"Slice value range: {np.min(sliced_img)} - {np.max(sliced_img)}")

sitk.WriteImage(sliced_volume, "../outputs/sliced_volume.nii.gz")

In [ ]:
sliced_volume_size = sliced_volume.GetSize()
sliced_img_size = sliced_img.shape

print(f" {sliced_volume_size=},\n {volume_size=},\n {sliced_img_size=},\n {img_size=}")

In [ ]:
print(origin)
origin_idx = volume.TransformPhysicalPointToIndex(origin)
print(origin_idx)
slice = sliced_img[:, 0, :]
plt.imshow(slice, aspect=6)
plt.scatter(origin_idx[0], origin_idx[2], c='r', marker='*', s=20)
plt.show()

In [ ]:
def slice_volume(z_rotation: float, x_rotation: float, translation: np.ndarray[4,4], volume: sitk.Image) -> (np.ndarray[4,4], sitk.Image):
    """
    Slice a 3D volume with arbitrary rotation and translation
    :param z_rotation: rotation around z-axis in degrees
    :param x_rotation: rotation around x-axis in degrees
    :param translation: translation vector
    :param volume: 3D volume to be sliced
    :return: Euler transformation matrix and the sliced volume
    """

    # Euler transformation
    # Rotation is defined by three rotations around z1, x2, z2 axis
    th_z1 = np.deg2rad(z_rotation)
    th_x2 = np.deg2rad(x_rotation)

    o = np.array(volume.GetOrigin())
    t = translation

    # transformation simplified at z2=0 since this rotation is never performed
    eul_tr = np.array([ [np.cos(th_z1), -np.sin(th_z1)*np.cos(th_x2),  np.sin(th_z1)*np.sin(th_x2), o[0]+t[0]],
                        [np.sin(th_z1),  np.cos(th_z1)*np.cos(th_x2), -np.cos(th_z1)*np.sin(th_x2), o[1]+t[1]],
                        [0,              np.sin(th_x2),                np.cos(th_x2),               o[2]+t[2]],
                        [0,              0,                            0,                           1]])

    # Define plane's coordinate system
    e1 = eul_tr[0][:3]
    e2 = eul_tr[1][:3]
    e3 = eul_tr[2][:3]
    img_o = eul_tr[:, -1:].flatten()[:3] # origin of the image plane

    direction = np.stack([e1, e2, e3], axis=0).flatten()

    resampler = sitk.ResampleImageFilter()
    spacing = volume.GetSpacing()
    volume_size = volume.GetSize()

    # Define the size of the output image
    # height of the image plane: original z size divided by cosine of x-rotation
    h = int(abs(volume_size[2]//e3[2]))
    # width of the image plane: original x size divided by cosine of z-rotation
    w = int(abs(volume_size[0]//e1[0]))

    resampler.SetOutputDirection(direction.tolist())
    resampler.SetOutputOrigin(img_o.tolist())
    resampler.SetOutputSpacing(spacing)
    resampler.SetSize((w, 3, h))
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)

    # Resample the volume on the arbitrary plane
    sliced_volume = resampler.Execute(volume)

    return eul_tr, sliced_volume

In [ ]:
sliced_volume = slice_volume(z_rotation=19.3, x_rotation=0, translation=np.array([0, 140, 0]), volume=volume)
sliced_img = sitk.GetArrayFromImage(sliced_volume[1])
print(f"Slice value range: {np.min(sliced_img)} - {np.max(sliced_img)}")

slice = sliced_img[:, 0, :]
plt.imshow(slice, aspect=6)
plt.scatter(origin_idx[0], origin_idx[2], c='r', marker='*', s=20)
plt.show()